In [ ]:
# create output files
import os
import h5py
import json
from tqdm import tqdm

import math

base_path = "./datasets/IndoorVG_4"
coco_path = os.path.join(base_path, "coco_anno")
rel_data = os.path.join(coco_path, "rel.json")
test_data = os.path.join(coco_path, "test.json")
train_data = os.path.join(coco_path, "train.json")
val_data = os.path.join(coco_path, "val.json")

input_h5 = os.path.join(base_path, "VG-SGG-augmented-penet-cat.h5")
input_h5 = h5py.File(input_h5, "r")
cat_path = os.path.join(base_path, "VG-SGG-dicts.json")
categories = json.load(open(cat_path, "r"))
image_data = "./datasets/VG150/image_data.json"
image_data = json.load(open(image_data, "r"))

In [19]:
print(im_to_first_box[1])

9


In [17]:
# Get box information
all_labels = input_h5['labels'][:, 0]
#all_attributes = roi_h5['attributes'][:, :]
BOX_SIZE= 1024
all_boxes = input_h5['boxes_{}'.format(BOX_SIZE)][:]  # cx,cy,w,h

# convert from xc, yc, w, h to x1, y1, x2, y2
# all_boxes[:, :2] = all_boxes[:, :2] - all_boxes[:, 2:] / 2
# all_boxes[:, 2:] = all_boxes[:, :2] + all_boxes[:, 2:]

im_to_first_box = input_h5['img_to_first_box']
im_to_last_box = input_h5['img_to_last_box']
im_to_first_rel = input_h5['img_to_first_rel']
im_to_last_rel = input_h5['img_to_last_rel']

# load relation labels
_relations = input_h5['relationships'][:]
_relation_predicates = input_h5['predicates'][:, 0]

print(len(_relation_predicates))

test_dict = {'images': [], 'annotations': [], 'categories': []}
train_dict = {'images': [], 'annotations': [], 'categories': []}
val_dict = {'images': [], 'annotations': [], 'categories': []}

rel_dict = {'train': {}, 'val': {}, 'test': {}, 'rel_categories': []}

for i in tqdm(range(len(im_to_first_box))):
    i_obj_start = im_to_first_box[i]
    i_obj_end = im_to_last_box[i]
    i_rel_start = im_to_first_rel[i]
    i_rel_end = im_to_last_rel[i]

    img_name = '{}.jpg'.format(image_data[i]['image_id'])
    img_id =image_data[i]['image_id']

    boxes_i = all_boxes[i_obj_start : i_obj_end + 1, :]
    gt_classes_i = all_labels[i_obj_start : i_obj_end + 1]

    split_i = input_h5['split_rel'][i]

    if split_i == 0:
        data_dict = train_dict
        rels = rel_dict['train']
    elif split_i == 1:
        data_dict = val_dict
        rels = rel_dict['val']
    else:
        data_dict = test_dict
        rels = rel_dict['test']

    for j in range(i_obj_start, i_obj_end + 1):
        box = all_boxes[j]
        # reshape box to img size using BOX_SIZE
        img_w, img_h = image_data[i]['width'], image_data[i]['height']
        box = box / BOX_SIZE * max(img_w, img_h)
        # to int
        box_orig = box
        # cx, cy to x1, y1
        box[0] = min(max(0, box[0] - (box[2] // 2)), img_w)
        box[1] = min(max(0, box[1] - (box[3] // 2)), img_h)

        if box[0] + box[2] > img_w:
            box[2] = img_w - box[0]

        if box[1] + box[3] > img_h:
            box[3] = img_h - box[1]
        box = box.astype(int)

        # verify that all coordinates are within the image size
        assert box[0] >= 0 & box[1] >= 0 & box[0] + box[2] <= img_w & box[1] + box[3] <= img_h, "Box coordinates are not within the image size : {}, width: {}, height: {} {} {}".format(box, img_w, img_h, img_name, box_orig)

        area = int(box[2] * box[3])
        data_dict['annotations'].append({'segmentation': [],
                                            'area': area,
                                            'iscrowd': 0,
                                            'image_id': img_id,
                                            'bbox': box.tolist(),
                                            'category_id': int(all_labels[j]),
                                            'id': j})
        
    rels[img_id] = []
    for j in range(i_rel_start, i_rel_end + 1):
        rel = _relations[j]
        rel_id = j
        subj_id = rel[0]-i_obj_start
        obj_id = rel[1]-i_obj_start
        rel_label = _relation_predicates[j]

        rels[img_id].append([int(subj_id), int(obj_id), int(rel_label)])
        
    data_dict['images'].append({'file_name': img_name,
                                'height': image_data[i]['height'],
                                'width': image_data[i]['width'],
                                'id': image_data[i]['image_id']})
    
# get rel categories
pred_labels = categories['predicate_to_idx']
rel_dict['rel_categories'] = ['__background__']
for pred, pred_id in pred_labels.items():
    rel_dict['rel_categories'].append(pred)

obj_labels = categories['label_to_idx']
for obj, obj_id in obj_labels.items():
    train_dict['categories'].append({'supercategory': obj,
                                        'id': obj_id,
                                        'name': obj})
    test_dict['categories'].append({'supercategory': obj,
                                        'id': obj_id,
                                        'name': obj})   
    val_dict['categories'].append({'supercategory': obj,
                                        'id': obj_id,
                                        'name': obj})

112804


100%|██████████| 108073/108073 [00:16<00:00, 6395.96it/s]


In [22]:
# save to json
with open(rel_data, 'w') as f:
    json.dump(rel_dict, f)

with open(test_data, 'w') as f:
    json.dump(test_dict, f)

with open(train_data, 'w') as f:
    json.dump(train_dict, f)

with open(val_data, 'w') as f:
    json.dump(val_dict, f)


In [21]:
file ="/home/maelic/Documents/Datasets/VG/annotations/rel.json"

import json
with open(file) as f:
    data = json.load(f)

print(data.keys())
for k in data.keys():
    print(k)
    print(type(data[k]))
    print(data[k]['498334'])
    # for q in data[k]:
    #     print(type(data[k][q]))
    #     # # print(data[k][q])
    #     break

    # print(len(data[k]))

dict_keys(['train', 'val', 'test', 'rel_categories'])
train
<class 'dict'>
[[1, 0, 20], [11, 8, 50], [12, 10, 31], [13, 5, 20], [13, 7, 21]]
val
<class 'dict'>


KeyError: '498334'